In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
#import missingno as msno
import requests
import geopandas
from mpl_toolkits.axes_grid1 import make_axes_locatable
import contextily as ctx
from shapely.geometry import Point

In [24]:
crime19 = pd.read_csv('datasets/crime19.csv')
crime19.rename(columns={'AREA ':'AREA'},inplace=True)
print(crime19.shape)
print(crime19.columns)

(2114699, 28)
Index(['DR_NO', 'Date Rptd', 'DATE OCC', 'TIME OCC', 'AREA', 'AREA NAME',
       'Rpt Dist No', 'Part 1-2', 'Crm Cd', 'Crm Cd Desc', 'Mocodes',
       'Vict Age', 'Vict Sex', 'Vict Descent', 'Premis Cd', 'Premis Desc',
       'Weapon Used Cd', 'Weapon Desc', 'Status', 'Status Desc', 'Crm Cd 1',
       'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4', 'LOCATION', 'Cross Street', 'LAT',
       'LON'],
      dtype='object')


In [25]:
crime20 = pd.read_csv('datasets/crime20.csv')
print(crime20.shape)
crime20.columns

(95405, 28)


Index(['DR_NO', 'Date Rptd', 'DATE OCC', 'TIME OCC', 'AREA', 'AREA NAME',
       'Rpt Dist No', 'Part 1-2', 'Crm Cd', 'Crm Cd Desc', 'Mocodes',
       'Vict Age', 'Vict Sex', 'Vict Descent', 'Premis Cd', 'Premis Desc',
       'Weapon Used Cd', 'Weapon Desc', 'Status', 'Status Desc', 'Crm Cd 1',
       'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4', 'LOCATION', 'Cross Street', 'LAT',
       'LON'],
      dtype='object')

#### Concatenate dataframes

In [26]:
crime = pd.concat([crime19,crime20],axis=0)
print(crime.shape)
crime.head().T

(2210104, 28)


,0,1,2,3,4
DR_NO,1307355,11401303,70309629,90631215,100100501
Date Rptd,02/20/2010 12:00:00 AM,09/13/2010 12:00:00 AM,08/09/2010 12:00:00 AM,01/05/2010 12:00:00 AM,01/03/2010 12:00:00 AM
DATE OCC,02/20/2010 12:00:00 AM,09/12/2010 12:00:00 AM,08/09/2010 12:00:00 AM,01/05/2010 12:00:00 AM,01/02/2010 12:00:00 AM
TIME OCC,1350,45,1515,150,2100
AREA,13,14,13,6,1
AREA NAME,Newton,Pacific,Newton,Hollywood,Central
Rpt Dist No,1385,1485,1324,646,176
Part 1-2,2,2,2,2,1
Crm Cd,900,740,946,900,122
Crm Cd Desc,VIOLATION OF COURT ORDER,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",OTHER MISCELLANEOUS CRIME,VIOLATION OF COURT ORDER,"RAPE, ATTEMPTED"


#### Reset index and convert lat-lon to shapely point features

In [29]:
crime['nu_index'] = list(range(0,crime.shape[0]))
crime = crime.set_index('nu_index')

In [30]:
crime['points'] = [Point(crime['LON'][i],crime['LAT'][i]) for i in range(crime.shape[0])]

### Maps

In [31]:
gdf = geopandas.read_file('../covid_la/datasets/gis_data/la-county-neighborhoods-current/l.a. county neighborhood (current).shp')
gdf.rename(columns={'name':'place'},inplace=True)
print(gdf.shape)
gdf.head(3)

(272, 11)


,slug,set,kind,external_i,place,display_na,sqmi,type,name_1,slug_1,geometry
0,acton,L.A. County Neighborhoods (Current),L.A. County Neighborhood (Current),acton,Acton,Acton L.A. County Neighborhood (Current),39.3391089485,unincorporated-area,None,None,"POLYGON ((-118.20262 34.53899, -118.18947 34.5..."
1,adams-normandie,L.A. County Neighborhoods (Current),L.A. County Neighborhood (Current),adams-normandie,Adams-Normandie,Adams-Normandie L.A. County Neighborhood (Curr...,0.805350187789,segment-of-a-city,None,None,"POLYGON ((-118.30901 34.03741, -118.30041 34.0..."
2,agoura-hills,L.A. County Neighborhoods (Current),L.A. County Neighborhood (Current),agoura-hills,Agoura Hills,Agoura Hills L.A. County Neighborhood (Current),8.14676029818,standalone-city,None,None,"POLYGON ((-118.76193 34.16820, -118.72632 34.1..."


In [32]:
dummy = [1 if crime['points'][i].within(gdf['geometry'][1]) else 0 for i in range(crime.shape[0])]

In [37]:
crime[gdf['place'][1]] = test

#### Loop through neighborhoods and dummy out by point within polygons (TAKES A WHILE)

In [39]:
for i in range(gdf.shape[0]):
    dummy = [1 if crime['points'][j].within(gdf['geometry'][i]) else 0 for j in range(crime.shape[0])]
    crime[gdf['place'][i]] = dummy

##### Use .sum(axis=0) to add up dummy 1s by neighborhood, access by index or by column name

In [40]:
crime.shape

(2210104, 301)

In [42]:
crime.to_csv('crime.csv')

In [2]:
test = geopandas.read_file('datasets/crime_maps.shp')
test.head().T

,0,1,2,3,4
slug,acton,adams-normandie,agoura-hills,agua-dulce,alhambra
set,L.A. County Neighborhoods (Current),L.A. County Neighborhoods (Current),L.A. County Neighborhoods (Current),L.A. County Neighborhoods (Current),L.A. County Neighborhoods (Current)
kind,L.A. County Neighborhood (Current),L.A. County Neighborhood (Current),L.A. County Neighborhood (Current),L.A. County Neighborhood (Current),L.A. County Neighborhood (Current)
external_i,acton,adams-normandie,agoura-hills,agua-dulce,alhambra
place,Acton,Adams-Normandie,Agoura Hills,Agua Dulce,Alhambra
display_na,Acton L.A. County Neighborhood (Current),Adams-Normandie L.A. County Neighborhood (Curr...,Agoura Hills L.A. County Neighborhood (Current),Agua Dulce L.A. County Neighborhood (Current),Alhambra L.A. County Neighborhood (Current)
sqmi,39.3391089485,0.805350187789,8.14676029818,31.4626319451,7.62381430605
type,unincorporated-area,segment-of-a-city,standalone-city,unincorporated-area,standalone-city
name_1,None,None,None,None,None
slug_1,None,None,None,None,None


### Downtown LA Focus

In [5]:
maps = geopandas.read_file('datasets/crime_maps.shp')
print(maps.shape)
maps.head()

(272, 45)


,slug,set,kind,external_i,place,display_na,sqmi,type,name_1,slug_1,...,change_13_,change_14_,change_15_,change_16_,change_17_,change_18_,change_19_,last10_pct,last5_pct_,geometry
0,acton,L.A. County Neighborhoods (Current),L.A. County Neighborhood (Current),acton,Acton,Acton L.A. County Neighborhood (Current),39.3391089485,unincorporated-area,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-13158255.282 4101406.864, -13156791..."
1,adams-normandie,L.A. County Neighborhoods (Current),L.A. County Neighborhood (Current),adams-normandie,Adams-Normandie,Adams-Normandie L.A. County Neighborhood (Curr...,0.805350187789,segment-of-a-city,None,None,...,0.013304,-0.063457,0.101636,0.073171,-0.012846,-0.011011,-0.046559,-0.006542,0.104391,"POLYGON ((-13170098.576 4033826.223, -13169141..."
2,agoura-hills,L.A. County Neighborhoods (Current),L.A. County Neighborhood (Current),agoura-hills,Agoura Hills,Agoura Hills L.A. County Neighborhood (Current),8.14676029818,standalone-city,None,None,...,-1.000000,NaN,inf,-1.000000,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-13220517.210 4051409.977, -13216553..."
3,agua-dulce,L.A. County Neighborhoods (Current),L.A. County Neighborhood (Current),agua-dulce,Agua Dulce,Agua Dulce L.A. County Neighborhood (Current),31.4626319451,unincorporated-area,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-13164050.576 4104017.289, -13164142..."
4,alhambra,L.A. County Neighborhoods (Current),L.A. County Neighborhood (Current),alhambra,Alhambra,Alhambra L.A. County Neighborhood (Current),7.62381430605,standalone-city,None,None,...,-0.176471,-0.500000,1.000000,-0.214286,0.636364,-0.444444,0.000000,0.346617,0.977633,"POLYGON ((-13149252.755 4042914.947, -13148709..."


In [ ]:
test = crime[(crime['Downtown']==1) & (crime['V']==1)]['points']